## Libraries

In [2]:
# import librariries
from bs4 import BeautifulSoup
from lxml import html
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm import tqdm
import json
import re

## Methods

In [2]:
def scrapingURL(url):
    driver = webdriver.Chrome()
    driver.get(url)

    content = driver.page_source
    soup = BeautifulSoup(content, 'html.parser')

    possible_recipe_urls = []
    for link in soup.find_all('a'):
        current_link = link.get('href')
        if current_link is None:
            continue
        if '/recipes/' in current_link:
            possible_recipe_urls.append(current_link)
    possible_recipe_urls = list(set(possible_recipe_urls))

In [27]:
def write2Json(recipes_ingre_dict, filename):
    # write data to json file
    json_object = json.dumps(recipes_ingre_dict, indent=4)
    
    # Writing to sample.json
    with open(filename, "w") as outfile:
        outfile.write(json_object)

## url: https://www.tasteofhome.com/collection/diabetic-cookie-recipes/


In [22]:
url = 'https://www.tasteofhome.com/collection/diabetic-cookie-recipes/'
driver = webdriver.Chrome()
driver.get(url)

content = driver.page_source
soup = BeautifulSoup(content, 'html.parser')

In [23]:
# all recipe links appeared on the webpage
possible_recipe_urls = []
for link in soup.find_all('a'):
    current_link = link.get('href')
    if current_link is None:
        continue
    if '/recipes/' in current_link:
        possible_recipe_urls.append(current_link)
possible_recipe_urls = list(set(possible_recipe_urls))
print(f'Number of available links: {len(possible_recipe_urls)}')
print(f'Stored the following links: {possible_recipe_urls}')

Number of available links: 32
Stored the following links: ['https://www.tasteofhome.com/recipes/vanilla-meringue-cookies/', 'https://www.tasteofhome.com/recipes/swirled-mint-cookies/', '/recipes/cooking-style/easy-recipes/', 'https://www.tasteofhome.com/recipes/dishes-beverages/', '/recipes/', 'https://www.tasteofhome.com/recipes/ingredients/', 'https://www.tasteofhome.com/recipes/banana-chocolate-chip-cookies/', 'https://www.tasteofhome.com/recipes/carrot-cookie-bites/', 'https://www.tasteofhome.com/recipes/techniques/', 'https://www.tasteofhome.com/recipes/cooking-style/', '/recipes/meal-types/dinner/', 'https://www.tasteofhome.com/recipes/chocolate-crinkle-cookies/', 'https://www.tasteofhome.com/recipes/chipotle-crackle-cookies/', 'https://www.tasteofhome.com/recipes/maple-walnut-biscotti/', 'https://www.tasteofhome.com/recipes/add', 'https://www.tasteofhome.com/recipes/cuisines/', 'https://www.tasteofhome.com/recipes/chewy-coconut-macaroons/', 'https://www.tasteofhome.com/recipes/c

In [26]:
# discard links that are not recipes
recipe_links = []
for link in possible_recipe_urls:
    if 'http' not in link:
        continue

    driver = webdriver.Chrome()
    driver.get(link)

    content = driver.page_source
    soup = BeautifulSoup(content, 'html.parser')

    if 'ingredients' in soup.text.lower():
        recipe_links.append(link)

In [29]:
recipe_links

['https://www.tasteofhome.com/recipes/vanilla-meringue-cookies/',
 'https://www.tasteofhome.com/recipes/swirled-mint-cookies/',
 'https://www.tasteofhome.com/recipes/dishes-beverages/',
 'https://www.tasteofhome.com/recipes/ingredients/',
 'https://www.tasteofhome.com/recipes/banana-chocolate-chip-cookies/',
 'https://www.tasteofhome.com/recipes/carrot-cookie-bites/',
 'https://www.tasteofhome.com/recipes/techniques/',
 'https://www.tasteofhome.com/recipes/cooking-style/',
 'https://www.tasteofhome.com/recipes/chocolate-crinkle-cookies/',
 'https://www.tasteofhome.com/recipes/chipotle-crackle-cookies/',
 'https://www.tasteofhome.com/recipes/maple-walnut-biscotti/',
 'https://www.tasteofhome.com/recipes/chewy-coconut-macaroons/',
 'https://www.tasteofhome.com/recipes/coconut-kisses/',
 'https://www.tasteofhome.com/recipes/meal-types/',
 'https://www.tasteofhome.com/recipes/chocolate-peanut-butter-cookies/',
 'https://www.tasteofhome.com/recipes/pumpkin-seed-cranberry-biscotti/',
 'https

In [66]:
# manually discard links that are not recipes
recipe_links.remove('https://www.tasteofhome.com/recipes/dishes-beverages/')
recipe_links.remove('https://www.tasteofhome.com/recipes/ingredients/')
recipe_links.remove('https://www.tasteofhome.com/recipes/techniques/')
recipe_links.remove('https://www.tasteofhome.com/recipes/cooking-style/')
recipe_links.remove('https://www.tasteofhome.com/recipes/meal-types/')
recipe_links.remove('https://www.tasteofhome.com/recipes/')
recipe_links.remove('https://www.tasteofhome.com/recipes/cooking-style/comfort-food/')

In [67]:
recipe_links

['https://www.tasteofhome.com/recipes/vanilla-meringue-cookies/',
 'https://www.tasteofhome.com/recipes/swirled-mint-cookies/',
 'https://www.tasteofhome.com/recipes/banana-chocolate-chip-cookies/',
 'https://www.tasteofhome.com/recipes/carrot-cookie-bites/',
 'https://www.tasteofhome.com/recipes/chocolate-crinkle-cookies/',
 'https://www.tasteofhome.com/recipes/chipotle-crackle-cookies/',
 'https://www.tasteofhome.com/recipes/maple-walnut-biscotti/',
 'https://www.tasteofhome.com/recipes/chewy-coconut-macaroons/',
 'https://www.tasteofhome.com/recipes/coconut-kisses/',
 'https://www.tasteofhome.com/recipes/chocolate-peanut-butter-cookies/',
 'https://www.tasteofhome.com/recipes/pumpkin-seed-cranberry-biscotti/',
 'https://www.tasteofhome.com/recipes/cinnamon-roll-macarons/',
 'https://www.tasteofhome.com/recipes/peppermint-kisses/',
 'https://www.tasteofhome.com/recipes/triple-ginger-gingersnaps/',
 'https://www.tasteofhome.com/recipes/molasses-crackle-cookies/',
 'https://www.tasteof

In [ ]:
recipes_ingre_dict = dict()
for link in tqdm(recipe_links):
    print(link)
    driver = webdriver.Chrome()
    driver.get(link)

    content = driver.page_source
    dom = html.fromstring(content)
    ingre_list = dom.xpath('//div[@class="recipe-ingredients"]//li//text()')
    
    recipes_ingre_dict[link.split('/')[-2]] = ingre_list

In [64]:
# write data to json file
json_object = json.dumps(recipes_ingre_dict, indent=4)
 
# Writing to sample.json
with open("tasteofhome.json", "w") as outfile:
    outfile.write(json_object)

## url: https://www.allrecipes.com/gallery/sugar-free-cookies/

In [8]:
url = 'https://www.allrecipes.com/gallery/sugar-free-cookies/'
driver = webdriver.Chrome()
driver.get(url)

content = driver.page_source
soup = BeautifulSoup(content, 'html.parser')

In [27]:
possible_recipe_urls = []
for i in soup.find_all('a'):
    if 'View Recipe' in str(i):
        possible_recipe_urls.append(i.get('href'))
possible_recipe_urls = list(set(possible_recipe_urls))
print(f'Number of available links: {len(possible_recipe_urls)}')
print(f'Stored the following links: {possible_recipe_urls}')

Number of available links: 13
Stored the following links: ['https://www.allrecipes.com/recipe/11594/chocolate-chip-cookies-for-special-diets/', 'https://www.allrecipes.com/recipe/261181/4-ingredient-keto-peanut-butter-cookies/', 'https://www.allrecipes.com/recipe/158403/sugar-free-peanut-butter-cookies/', 'https://www.allrecipes.com/recipe/10905/no-bake-bumpy-peanut-butter-nuggets/', 'https://www.allrecipes.com/recipe/233511/paleo-coconut-dark-chocolate-chip-cookies/', 'https://www.allrecipes.com/recipe/275260/sugar-free-date-cookies/', 'https://www.allrecipes.com/recipe/50441/pumpkin-protein-cookies/', 'https://www.allrecipes.com/recipe/261183/chewy-keto-chocolate-cookies/', 'https://www.allrecipes.com/recipe/11349/no-sugar-added-cookies/', 'https://www.allrecipes.com/recipe/218490/low-carb-flavored-meringue-cookies/', 'https://www.allrecipes.com/recipe/261182/low-carb-almond-cinnamon-butter-cookies/', 'https://www.allrecipes.com/recipe/19202/no-sugar-raisin-cookies/', 'https://www.al

In [31]:
link = 'https://www.allrecipes.com/recipe/11594/chocolate-chip-cookies-for-special-diets/'
driver = webdriver.Chrome()
driver.get(link)

content = driver.page_source
dom = html.fromstring(content)

In [56]:
recipes_ingre_dict = dict()
for link in tqdm(possible_recipe_urls):
    print(link)
    driver = webdriver.Chrome()
    driver.get(link)

    content = driver.page_source
    dom = html.fromstring(content)
    token_ingre_list = dom.xpath('//div[@class="comp mntl-lrs-ingredients mntl-block"]//li//text()')

    rebuild_ingre_list = []
    current_ingredient = ''
    for ingre in token_ingre_list:
        if ingre != '\n':
            current_ingredient += str(ingre)
        else:
            if ',' in current_ingredient:
                current_ingredient = current_ingredient.replace(',', '')
            current_ingredient = current_ingredient.replace('  ', ' ')
            rebuild_ingre_list.append(current_ingredient)
            current_ingredient = ''

    for ingre in rebuild_ingre_list:
        if ingre == '':
            rebuild_ingre_list.remove(ingre)
    
    recipes_ingre_dict[link.split('/')[-2]] = rebuild_ingre_list

  0%|          | 0/13 [00:00<?, ?it/s]

https://www.allrecipes.com/recipe/11594/chocolate-chip-cookies-for-special-diets/


  8%|▊         | 1/13 [00:04<00:51,  4.25s/it]

https://www.allrecipes.com/recipe/261181/4-ingredient-keto-peanut-butter-cookies/


 15%|█▌        | 2/13 [00:08<00:46,  4.27s/it]

...

 85%|████████▍ | 11/13 [00:45<00:08,  4.10s/it]

https://www.allrecipes.com/recipe/19202/no-sugar-raisin-cookies/


 92%|█████████▏| 12/13 [00:49<00:04,  4.06s/it]

https://www.allrecipes.com/recipe/15231/special-oatmeal-cookies/


100%|██████████| 13/13 [00:53<00:00,  4.11s/it]


In [64]:
recipes_ingre_dict

{'chocolate-chip-cookies-for-special-diets': ['½ cup butter softened',
  '¾ cup granulated artificial sweetener',
  '2 tablespoons water',
  '½ teaspoon vanilla extract',
  '1 egg beaten',
  '1 ⅛ cups all-purpose flour',
  '½ teaspoon baking soda',
  '½ teaspoon salt',
  '½ cup semisweet chocolate chips',
  '½ cup chopped pecans'],
 '4-ingredient-keto-peanut-butter-cookies': ['1 cup peanut butter',
  '½ cup low-calorie natural sweetener (such as Swerve®)',
  '1 egg',
  '1 teaspoon sugar-free vanilla extract'],
 'sugar-free-peanut-butter-cookies': ['2 cups smooth natural peanut butter',
  '2 cups granular no-calorie sucralose sweetener (e.g. Splenda ®)',
  '2 large eggs'],
 'no-bake-bumpy-peanut-butter-nuggets': ['½ cup natural peanut butter',
  '¼ cup nonfat dry milk powder',
  '¼ cup unsweetened flaked coconut',
  '⅓ cup rolled oats',
  '½ teaspoon ground cinnamon',
  '¼ cup wheat germ',
  '¼ cup unsweetened apple juice concentrate thawed'],
 'paleo-coconut-dark-chocolate-chip-cookies

In [58]:
write2Json(
    recipes_ingre_dict=recipes_ingre_dict,
    filename='allrecipes.json'
)

## url: https://www.everydaydiabeticrecipes.com/Cookie-Recipes/Diabetic-Cookie-Recipes-Top-Best-Cookie-Recipes-Youll-Love

In [6]:
url = 'https://www.everydaydiabeticrecipes.com/Cookie-Recipes/Diabetic-Cookie-Recipes-Top-Best-Cookie-Recipes-Youll-Love/'
base = 'https://www.everydaydiabeticrecipes.com'
driver = webdriver.Chrome()
driver.get(url)

content = driver.page_source
soup = BeautifulSoup(content, 'html.parser')

In [7]:
myset = set()
for token in soup.find_all('a'):
    if '/Cookie-Recipes/' in str(token):
        myset.add(token.get('href'))
print('Recipe lins found: ', len(myset))
myset = list(myset)
for i in myset:
    print(i)

Recipe lins found:  18
/Cookie-Recipes/Spiced-Pumpkin-Chip-Cookies
/Cookie-Recipes/Pumpkin-Pie-Cookies
/Cookie-Recipes/Grandmas-Oatmeal-Raisin-Cookies
/Cookie-Recipes/Chocolate-Chip-Carrot-Cookies-EDR
/Cookie-Recipes/Ginger-Spice-Chocolate-Cookies
/Cookie-Recipes/Lickety-Split-Lemon-Oaties-6652
/Cookie-Recipes/Diabetic-Cookie-Recipes-Top-Best-Cookie-Recipes-Youll-Love
/Cookie-Recipes/Fruity-Fingerprint-Cookies
/Cookie-Recipes/Oatmeal-Chocolate-Chip-Cookies-204976
/Cookie-Recipes/Chocolate-Chip-Cloud-Cookies
/Cookie-Recipes/Snickerdoodles
/Cookie-Recipes/Fudgy-Chocolate-Cookies
/Cookie-Recipes/Banana-Bread-Cookies
/Cookie-Recipes/Pecan-Balls
/Cookie-Recipes/Brookies
/Cookie-Recipes/Black-and-White-Cookies-EDR
/Cookie-Recipes/3-Ingredient-Peanut-Butter-Cookies
/Cookie-Recipes/Cinnamon-Roll-Cookies-204986


In [18]:
recipes_ingre_dict = dict()
for l in tqdm(myset):
    link = base + l
    print(link)
    driver = webdriver.Chrome()
    driver.get(link)

    content = driver.page_source
    dom = html.fromstring(content)
    ingre_list = dom.xpath('//div[@class="recipeIngredients"]//li//text()')
    recipes_ingre_dict[l.split('/')[-1]] = ingre_list

  0%|          | 0/18 [00:00<?, ?it/s]

https://www.everydaydiabeticrecipes.com/Cookie-Recipes/Spiced-Pumpkin-Chip-Cookies


  6%|▌         | 1/18 [00:06<01:53,  6.67s/it]

https://www.everydaydiabeticrecipes.com/Cookie-Recipes/Pumpkin-Pie-Cookies


 11%|█         | 2/18 [00:13<01:44,  6.53s/it]

...

 89%|████████▉ | 16/18 [02:07<00:15,  7.66s/it]

https://www.everydaydiabeticrecipes.com/Cookie-Recipes/3-Ingredient-Peanut-Butter-Cookies


 94%|█████████▍| 17/18 [02:14<00:07,  7.56s/it]

https://www.everydaydiabeticrecipes.com/Cookie-Recipes/Cinnamon-Roll-Cookies-204986


100%|██████████| 18/18 [02:20<00:00,  7.82s/it]


In [19]:
write2Json(
    recipes_ingre_dict=recipes_ingre_dict,
    filename='everydaydiabetic.json'
)

## url: https://www.eatingwell.com/recipes/19967/health-condition/diabetic/desserts/cookies-bars-brownies/

In [20]:
url = 'https://www.eatingwell.com/recipes/19967/health-condition/diabetic/desserts/cookies-bars-brownies/'
driver = webdriver.Chrome()
driver.get(url)

content = driver.page_source
soup = BeautifulSoup(content, 'html.parser')

In [29]:
recipe_list = []
for token in soup.find_all('a'):
    if '/recipe/' in str(token):
        recipe_list.append(token.get('href'))

In [40]:
recipes_ingre_dict = dict()
for link in tqdm(recipe_list):
    print(link)
    driver = webdriver.Chrome()
    driver.get(link)

    content = driver.page_source
    dom = html.fromstring(content)
    token_ingre_list = dom.xpath('//div[@class="comp mntl-lrs-ingredients mntl-block"]//li//text()')

    rebuild_ingre_list = []
    current_ingredient = ''
    for ingre in token_ingre_list:
        if ingre != '\n':
            current_ingredient += str(ingre)
        else:
            if ',' in current_ingredient:
                current_ingredient = current_ingredient.replace(',', '')
            current_ingredient = current_ingredient.replace('  ', ' ')
            rebuild_ingre_list.append(current_ingredient)
            current_ingredient = ''

    for ingre in rebuild_ingre_list:
        if ingre == '':
            rebuild_ingre_list.remove(ingre)
    
    recipes_ingre_dict[link.split('/')[-2]] = rebuild_ingre_list

  0%|          | 0/42 [00:00<?, ?it/s]

https://www.eatingwell.com/recipe/261075/no-sugar-added-vegan-oatmeal-cookies/


  2%|▏         | 1/42 [00:03<02:36,  3.81s/it]

https://www.eatingwell.com/recipe/277433/peanut-butter-chocolate-chip-cookies/


  5%|▍         | 2/42 [00:07<02:34,  3.87s/it]

...

 95%|█████████▌| 40/42 [02:32<00:07,  3.85s/it]

https://www.eatingwell.com/recipe/7877683/chocolate-coconut-thumbprints/


 98%|█████████▊| 41/42 [02:36<00:03,  3.84s/it]

https://www.eatingwell.com/recipe/8016792/lemon-poppy-seed-hamantaschen/


100%|██████████| 42/42 [02:40<00:00,  3.81s/it]


In [41]:
write2Json(
    recipes_ingre_dict=recipes_ingre_dict,
    filename='eatingwell.json'
)

## Preprocessing

In [3]:
f = open('everydaydiabetic.json')
data = json.load(f)
f.close()

In [29]:
def has_numbers(inputString):
    return bool(re.search(r'\d', inputString))

def format_recipe(current_list):
    output_list = []
    for i in range(len(current_list)):
        output_list.append(current_list[i].strip())
    # print(output_list)

    output_final = []
    current_ingr = ''
    i=0
    while i<len(output_list):
        if output_list[i].isnumeric():
            current_ingr += output_list[i] + ' '
        elif output_list[i] == '/':
            current_ingr += output_list[i] + ' '
        elif isinstance(output_list[i],str) and has_numbers(output_list[i]):
            output_final.append(current_ingr)
            output_final.append(output_list[i])
            current_ingr = ''
        elif isinstance(output_list[i],str) and not has_numbers(output_list[i]):
            current_ingr += output_list[i] 
            output_final.append(current_ingr)
            current_ingr = ''
        else:
            print('there is a problem')
        i+=1

    # remove empty entries in list
    output_final = [token for token in output_final if token]

    # remove unnecessary spaces
    output_final = [token.replace(' / ', '/') for token in output_final]

    return output_final

dict_keys = list(data.keys())
backup_dict = dict()
for key in dict_keys:
    backup_dict[key] = format_recipe(data[key]) 

In [30]:
write2Json(
    recipes_ingre_dict=backup_dict,
    filename='everydaydiabetic_new.json'
)